In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## 예측하려는 target값이 범주형 데이터면 분류, 수치형 데이터면 회귀

# 회귀

### 영향을 미치는 변수를 독립변수 영향을 받는 변수를 종속 변수
### 회귀는 독립변수와 종속변수 간 관계를 리모델링하는 방법
### 분류와 차이점은 타깃 데이터가 범주가 아닌 수치형 데이터
### 주어진 독립변수(피처)와 종속변수(타깃값) 사이의 관계를 기반으로 최적의 회귀계수를 찾아야 됨

# 회귀의 평가 지표 
### 오차를 최소화 해야됨 = 평가지표의 값이 작을 수록 모델 성능이 좋음
### * R^2는 1에 가까울수록 좋음

### metrics 라이브러리를 활용해 회귀 평가 지표 값을 구해봄

In [2]:
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_squared_log_error, r2_score

true = np.array([1,2,3,2,3,5,4,6,5,6,7,8,8]) # 실제값
preds = np.array([1,1,2,2,3,4,4,5,5,7,7,6,8]) # 예측값

MAE = mean_absolute_error(true, preds)
MSE = mean_squared_error(true, preds)
RMSE = np.sqrt(MSE)
MSLE = mean_squared_log_error(true, preds)
RMSLE = np.sqrt(mean_squared_log_error(true, preds))
R2 = r2_score(true,preds)

print(round(MAE,4))
print(round(MSE,4))
print(round(RMSE,4))
print(round(MSLE,4))
print(round(RMSLE,4))
print(round(R2,4))

0.5385
0.6923
0.8321
0.0296
0.1721
0.8617


# 상관 계수
### 두 상관관계 정도를 수치로 나타낸 값을 상관 계수라고 함
### 상관계수는 보통 약자 r로 표현함
### 가장 많이 사용한는 상관계수는 피어슨 상관계수임.
### 선형 상관관계의 강도와 방향을 나타냄 -1 ~ 1 사이의 값을 갖음 상관계수가 음수면 음의 상관관계 양수면 양의 상관관계가 있다고 함

# 분류 평가지표
### 기본적인 지표 : 오차 행렬, 로그 손실, ROC 곡선, AUC

# 오차 행렬 : confusion matrix (이전에 배운 딥러닝 부분 참고)
### 정확도는 잘 사용 안함
### 정밀도 = TP/TP+FP
### 재현율 = TP/TP+FN // 민감도 또는 참 양성 비율(TPR)이라고도 함
### F1 점수는 정밀도와 재현율을 조합한 평기지표(해당책 116p 참고)

# 로그 손실
### 분류 문제에서 타깃값을 확률로 예측할 때 기본적으로 사용하는 평가지표(116p참고)


# ROC AUC 
### 곡선이 ROC, ROC 곡선 아래 면적이 AUC 가운데 점선에 ROC가 가까울수록 성능이 낮음(무작위로 분류함 완벽이 1 무작위가 0.5, 성능이 좋을수록 AUC(면적)이 큼)
### 캐글에서 종종사용하는 평가 지표

# 데이터 인코딩

### 문자로 구성된 범주형 데이터는 숫자로 바꾸어야 됨
### 경우에 따라서는 이미 숫자로 구성된 범주형 데이터도 모델 성능을 향상하기위해 다른 숫자 데이터로 바꾸기도함
### 범주형 데이터 -> 숫자형 데이터 = 데이터 인코딩
### 인코딩 방식 : 레이블 인코딩, 원-핫 인코딩

## 레이블 인코딩
### 범주형 데이터를 숫자로 일대일 매핑해주는 인코딩 방식 (범주형 데이터를 숫자형 데이터로 치환)

### 레이블 인코딩 적용(원본 데이터의 값에 사전순으로 번호를 매김)
### 단점 : 명목형 데이터를 레이블 인코딩하면 모델 성능이 떨어질 수 있음(머신러닝 모델이 서로 가까운 숫자를 비슷한 데이터라고 판단하기 때문)

In [3]:
from sklearn.preprocessing import LabelEncoder

fruits = ['사과', '블루베리', '바나나','귤','블루베리','바나나','바나나','사과']

label_encoding = LabelEncoder()

fruits_label_encoding = label_encoding.fit_transform(fruits)

print(fruits_label_encoding)

[3 2 1 0 2 1 1 3]


# 원-핫 인코딩
### 1. 인코딩하려는 피처의 고윳값 개수를 구함
### 2. 피처의 고윳값 개수만큼 열을 추가
### 3. 각 고윳값에 해당하는 열에 1을 표시하고 나머지 열에는 0을 표시
## 단점 : 열 개수가 지나치게 많아짐(모델 훈련속도가 느려질 우려가 있음)

In [4]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

fruits = ['사과', '블루베리', '바나나','귤','블루베리','바나나','바나나','사과']

encoding_label = LabelEncoder()
encoding_OneHot = OneHotEncoder()

fruits_encoding_label = encoding_label.fit_transform(fruits)
fruits_encoding_OneHot = encoding_OneHot.fit_transform(fruits_encoding_label.reshape(-1,1))

print(fruits_encoding_OneHot.toarray())

[[0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]]


In [5]:
import pandas as pd

fruits = ['사과', '블루베리', '바나나','귤','블루베리','바나나','바나나','사과']

pd.get_dummies(fruits) # pd.get_dummies : 원핫 인코딩한 결과값이 같음

,귤,바나나,블루베리,사과
0,0,0,0,1
1,0,0,1,0
2,0,1,0,0
3,1,0,0,0
4,0,0,1,0
5,0,1,0,0
6,0,1,0,0
7,0,0,0,1


# 피처 스케일링
### 서로 다른 피처 값(최댓값 - 최솟값)의 범위가 일치하도록 조정하는 작업을 말함
### 스케일링이 제대로 안되면 모델의 훈련이 정상적으로 안될 수 있음

### min-max 정규화
### 피처의 값을 0~1로 조절하는 기법 // 조정 후 최솟값은 0 최댓값은 1 
### 피처 x - min / max - min
### ** 이상치(대부분의 값과 동떨어진 값)가 너무 크거나 작을 때는 min-max 정규화가 좋지 않은 결과를 낼 수 있음(이상치가 너무 크거나 작을 땐 표준화가 더 바람직)

In [6]:
# min - max 정규화 구현
import pandas as pd

height_weight_dict = {'키': [1.7, 1.5, 1.8], '몸무게' : [75,55,60]}
df = pd.DataFrame(height_weight_dict, index = ['광일','혜성','덕수'])

print(df)

      키  몸무게
광일  1.7   75
혜성  1.5   55
덕수  1.8   60


In [7]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

scaler.fit(df)
df_scaler = scaler.transform(df)

print(df_scaler)

[[0.66666667 1.        ]
 [0.         0.        ]
 [1.         0.25      ]]


In [8]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

df_scaler = scaler.fit_transform(df) # fit 과 transform을 한번에 실행 가능함 하지만 위에서 두개로 나눈 이유는 다른 데이터에도 적용 하려면 하나의 데이터로 훈련 시킨후 다른 데이터들은 transform만 하면 편리하기 때문임

print(df_scaler)

[[0.66666667 1.        ]
 [0.         0.        ]
 [1.         0.25      ]]


# 표준화
### 평균이 0 분산이 1이 되도록 피처값을 조정하는 기법 // min-max 기법과 달리 상한과 하한이 없음
### 상한과 하한이 필요 없는 데이터에 적용 가능(정규분포를 따르는 데이터는 표준화로 스케일링을 적용하는것이 좋음)

In [9]:
# 표준화 구현
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()

ss_scaled = ss.fit_transform(df)

print(ss_scaled)

[[ 0.26726124  1.37281295]
 [-1.33630621 -0.98058068]
 [ 1.06904497 -0.39223227]]


# 교차 검증
### 일반적으로는 K 폴드 교차 검증 사용
### 1. 전체 데이터를 k개로 나눔
### 2. 그룹하나는 검증데이터 나머지 k-1개는 훈련 데이터로 지정
### 3. 훈련 데이터로 모델을 훈련 검증데이터로 평가
### 4. 평가점수 기록
### 5. 검증 데이터를 다른 그룹으로 바꿔가며 2~4절차를 K번 반복
### 6. K개 검증 평가 점수의 평균을 구함
## K개 검증 평가점수의 평균이 최종 평가점수 이며 제출하지 않고도 이점수를 통해 모델 성능 가늠해볼 수 있음

In [10]:
import numpy as np
from sklearn.model_selection import KFold

data = np.array([0,1,2,3,4,5,6,7,8,9])

folds = KFold(n_splits=5, shuffle=False) # KFold(): 데이터를 K 폴드로 나누는 함수 // n_split=K: parameter에 전달하는 값(K)

for train_idx, valid_idx in folds.split(data):
    print("훈련데이터 {}, 검증 데이터 : {}".format(train_idx, valid_idx))

훈련데이터 [2 3 4 5 6 7 8 9], 검증 데이터 : [0 1]
훈련데이터 [0 1 4 5 6 7 8 9], 검증 데이터 : [2 3]
훈련데이터 [0 1 2 3 6 7 8 9], 검증 데이터 : [4 5]
훈련데이터 [0 1 2 3 4 5 8 9], 검증 데이터 : [6 7]
훈련데이터 [0 1 2 3 4 5 6 7], 검증 데이터 : [8 9]


In [11]:
import numpy as np
from sklearn.model_selection import KFold

data = np.array([0,1,2,3,4,5,6,7,8,9])

folds = KFold(n_splits=5, shuffle=True) # shuffle = boolean : 데이터가 편향되어있을 수 있기 때문에 나누기 전에 섞어줌

for train_idx, valid_idx in folds.split(data):
    print("훈련데이터 {}, 검증 데이터 : {}".format(train_idx, valid_idx))

훈련데이터 [0 1 3 4 5 6 7 9], 검증 데이터 : [2 8]
훈련데이터 [0 1 2 4 6 7 8 9], 검증 데이터 : [3 5]
훈련데이터 [1 2 3 4 5 7 8 9], 검증 데이터 : [0 6]
훈련데이터 [0 1 2 3 5 6 8 9], 검증 데이터 : [4 7]
훈련데이터 [0 2 3 4 5 6 7 8], 검증 데이터 : [1 9]


# 층화 K 폴드 교차 검증(분류 문제에만 사용// 회귀는 균등하게 나눌 수 없기 때문에 사용 불가)
### 타깃값이 골고루 분포되게 폴드를 나누는 K폴드 교차 검증 방법
### 특정 타깃 값이 다른 타깃 값보다 굉장히 적은 경우 사용(이책의 133p 참고)

## 타깃값이 불균형하게 분포되어 있는 경우 층화 K 폴드 사용

##### KFold의 split()에는 데이터 하나만 전달해도 됨 데이터 불균형과 상관없이 임의로 K개로 분할하기 때문
##### 반면 StratifiedKFold의 split()에는 피처와 타깃 모두 전달해야됨 // 인수를 하나만 전달하면 오류 발생

In [12]:
# 층화 K 폴드 교차 검증 구현
import numpy as np

y = np.array(['스팸']*5 + ['일반']*45)

folds = KFold(n_splits=5 ,shuffle=True)

for idx, (train_idx, valid_idx) in enumerate(folds.split(y)): # 타깃값들이 골고루 분포되어있지않아 학습이 일어나지 않는 모델이 있음
    print("Fold {} 검증 데이터의 타깃값:".format(idx+1)) 
    print(y[valid_idx],"\n")

Fold 1 검증 데이터의 타깃값:
['스팸' '일반' '일반' '일반' '일반' '일반' '일반' '일반' '일반' '일반'] 

Fold 2 검증 데이터의 타깃값:
['스팸' '일반' '일반' '일반' '일반' '일반' '일반' '일반' '일반' '일반'] 

Fold 3 검증 데이터의 타깃값:
['스팸' '일반' '일반' '일반' '일반' '일반' '일반' '일반' '일반' '일반'] 

Fold 4 검증 데이터의 타깃값:
['일반' '일반' '일반' '일반' '일반' '일반' '일반' '일반' '일반' '일반'] 

Fold 5 검증 데이터의 타깃값:
['스팸' '스팸' '일반' '일반' '일반' '일반' '일반' '일반' '일반' '일반'] 



In [13]:
from sklearn.model_selection import StratifiedKFold

x = np.array(range(50))
y = np.array(['스팸']*5 + ['일반']*45)

folds = StratifiedKFold(n_splits=5) # 층화 K 폴드 교차 검증 // 모든 모델에 target이 포함됨

for idx, (train_idx, valid_idx) in enumerate(folds.split(x,y)): 
    print("Fold {} 검증 데이터의 타깃값:".format(idx+1)) 
    print(y[valid_idx],"\n")

Fold 1 검증 데이터의 타깃값:
['스팸' '일반' '일반' '일반' '일반' '일반' '일반' '일반' '일반' '일반'] 

Fold 2 검증 데이터의 타깃값:
['스팸' '일반' '일반' '일반' '일반' '일반' '일반' '일반' '일반' '일반'] 

Fold 3 검증 데이터의 타깃값:
['스팸' '일반' '일반' '일반' '일반' '일반' '일반' '일반' '일반' '일반'] 

Fold 4 검증 데이터의 타깃값:
['스팸' '일반' '일반' '일반' '일반' '일반' '일반' '일반' '일반' '일반'] 

Fold 5 검증 데이터의 타깃값:
['스팸' '일반' '일반' '일반' '일반' '일반' '일반' '일반' '일반' '일반'] 

